In [1]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

# Tools for assessing the quality of model prediction
from sklearn.metrics import accuracy_score, confusion_matrix
# load the dataset
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [7]:
df= pd.read_csv('/content/IMDB Dataset.csv',engine="python",error_bad_lines=False)
df.head()

Skipping line 22152: unexpected end of data


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
df['sentiment'] = df['sentiment'].apply(lambda x:1 if x=='negative' else 0)

In [9]:
df.head()
print(df)

                                                  review  sentiment
0      One of the other reviewers has mentioned that ...          0
1      A wonderful little production. <br /><br />The...          0
2      I thought this was a wonderful way to spend ti...          0
3      Basically there's a family where a little boy ...          1
4      Petter Mattei's "Love in the Time of Money" is...          0
...                                                  ...        ...
22145  Rich, alcoholic Robert Stack falls in love wit...          0
22146  In an era of such awful cartoons, I am rather ...          0
22147  Well, at least this was the last sequel that I...          1
22148  Oh my God... where to begin? "Chupacabra Terro...          1
22149  This movie is extremely boring, it tells a sto...          1

[22150 rows x 2 columns]


In [11]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
nltk.download('stopwords')
stopword_list=nltk.corpus.stopwords.words('english')

#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
df['review']=df['review'].apply(denoise_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
#function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#function on review column
df['review']=df['review'].apply(remove_special_characters)

In [13]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
df['review']=df['review'].apply(remove_stopwords)

{'wouldn', 'y', 'you', 's', "didn't", 'o', 'which', 'themselves', 'doesn', 'yours', "don't", 'weren', 'between', 'herself', 'itself', 'a', 'haven', "it's", 'as', 'll', 'should', 'i', 'an', 'yourselves', 'nor', 'too', 'both', 'other', "that'll", 'there', 'each', "hadn't", 'hers', 'being', 'through', 'only', 'under', 'aren', 're', 'them', 'does', 'had', 'when', 'do', 'his', 'himself', 'with', 'no', 'below', 'until', 'have', 'few', "shan't", 'him', 'been', 'he', 'to', 'did', 'yourself', 'couldn', "couldn't", "she's", 'at', 'before', "won't", 'in', "weren't", "wasn't", 'd', "doesn't", 'after', 'on', 'from', 'very', "wouldn't", 'mustn', 'that', 'these', "should've", 'myself', 'shouldn', 'such', 'didn', 'won', 't', 'own', 'out', 'once', 'and', "shouldn't", 'during', 'all', 'she', 'so', 'because', 'their', 'again', 'we', "isn't", 'down', "needn't", 'more', 'who', 'your', 'm', 'theirs', 'some', 'into', 'where', 'or', 'whom', 'was', 'off', 'be', 'any', 'than', 'is', "haven't", 'up', 'most', 'my

In [14]:
def load_dataset():
    df = pd.read_csv('/content/IMDB Dataset.csv',engine="python",error_bad_lines=False)
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in stop])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Skipping line 38818: unexpected end of data


Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
38811    [if, ever, watched, dukes, hazard, know, never...
38812    [this, film, basically, velvet, goldmine, writ...
38813    [the, centerpiece, lackawanna, blues, characte...
38814    [i, really, hope, makers, movies, read, review...
38815    [this, movie, like, happiness, meets, lost, tr...
Name: review, Length: 38816, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
38811    0
38812    0
38813    1
38814    0
38815    1
Name: sentiment, Length: 38816, dtype: int64


In [15]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)


In [16]:
print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
13427    [since, i, seen, three, i, figured, i, might, ...
25543    [movie, small, town, equal, numbers, mormons, ...
13724    [this, movie, really, bad, nothing, appealing,...
17006    [i, saw, original, black, white, stars, shelle...
32187    [leslie, charteris, series, novels, adventures...
                               ...                        
31333    [when, attempt, made, assassinate, emir, ohtar...
15306    [i, never, expected, old, film, impressive, we...
14026    [this, attempt, author, edgar, rice, burroughs...
3253     [the, plot, in, mood, love, simple, enough, a,...
3964     [beautiful, art, direction, excellent, editing...
Name: review, Length: 31052, dtype: object 

30874    [this, film, provides, us, interesting, remind...
17757    [i, got, plus, year, old, computer, games, bet...
13261    [i, like, film, i, think, much, book, either, ...
14770    [this, film, one, nostalgia, things, i, never,...
9460     [critters, this, movie, continued, rd, critter...
 

In [17]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [18]:
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()
#Each reviews has a different length, so we need to add padding (by adding 0) or truncating the
#words to the same length (in this case, it is the mean of all reviews length) using tensorflow.keras.preprocessing.sequence.pad_sequences.
x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[ 135    1   38 ...    1  372  190]
 [   3  307  401 ...    0    0    0]
 [   9    3   15 ...    0    0    0]
 ...
 [   9  518 1882 ...    0    0    0]
 [   2   40   49 ...    0    0    0]
 [ 214  413  373 ...    0    0    0]] 

Encoded X Test
 [[   9    4 1525 ...    0    0    0]
 [   1  102  827 ...    0    0    0]
 [   1    6    4 ...    0    0    0]
 ...
 [   2  704   40 ... 4287 4332    2]
 [   1   62  357 ...    0    0    0]
 [ 302  220 1918 ...    0    0    0]] 

Maximum review length:  130


In [19]:
# ARCHITECTURE
dim = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(words, dim, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 130, 32)           2665408   
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2,690,305
Trainable params: 2,690,305
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
callback = ModelCheckpoint(
    'models/imdb.h1',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [21]:
model.fit(x_train, y_train, batch_size = 32, epochs = 5, callbacks=[callback])

Epoch 1/5
971/971 [==============================] - ETA: 0s - loss: 0.4811 - accuracy: 0.7591
Epoch 00001: accuracy improved from -inf to 0.75908, saving model to models/imdb.h1
Cause: Unable to locate the source code of <function canonicalize_signatures.<locals>.signature_wrapper at 0x7f62a6861f80>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function canonicalize_signatures.<locals>.signature_wrapper at 0x7f62a6861f80>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the ca

INFO:tensorflow:Assets written to: models/imdb.h1/assets


INFO:tensorflow:Assets written to: models/imdb.h1/assets


971/971 [==============================] - 116s 117ms/step - loss: 0.4811 - accuracy: 0.7591
Epoch 2/5
971/971 [==============================] - ETA: 0s - loss: 0.2378 - accuracy: 0.9153
Epoch 00002: accuracy improved from 0.75908 to 0.91527, saving model to models/imdb.h1
Cause: Unable to locate the source code of <function canonicalize_signatures.<locals>.signature_wrapper at 0x7f62a657cb00>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function canonicalize_signatures.<locals>.signature_wrapper at 0x7f62a657cb00>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Cause: Unable to locate the source code of <function canonicalize_signatures.<locals>.signature_wrapper at 0x7f62a23730e0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function canonicalize_signatures.<locals>.signature_wrapper at 0x7f62a23730e0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


INFO:tensorflow:Assets written to: models/imdb.h1/assets


INFO:tensorflow:Assets written to: models/imdb.h1/assets


971/971 [==============================] - 113s 117ms/step - loss: 0.1165 - accuracy: 0.9675
Epoch 5/5
971/971 [==============================] - ETA: 0s - loss: 0.2828 - accuracy: 0.9045
Epoch 00005: accuracy did not improve from 0.96747
971/971 [==============================] - 109s 112ms/step - loss: 0.2828 - accuracy: 0.9045


In [22]:
scores = model.evaluate(x_test, y_test, verbose=1)
scores[1]


243/243 [==============================] - 6s 21ms/step - loss: 0.5844 - accuracy: 0.8026


0.8025502562522888

In [23]:
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 80.26%


In [24]:
predict_x = model.predict(x_test)
classes_x = np.argmax(predict_x,axis=1)



In [25]:
loaded_model = load_model('models/imdb.h1')

In [26]:
review = str(input('Movie Review: '))

Movie Review: Nothing was typical about this. Everything was beautifully done in this movie, the story, the flow, the scenario, everything. I highly recommend it for mystery lovers, for anyone who wants to watch a good movie!


In [28]:
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in stop]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  Nothing was typical about this Everything was beautifully done in this movie the story the flow the scenario everything I highly recommend it for mystery lovers for anyone who wants to watch a good movie
Filtered:  ['nothing typical everything beautifully done movie story flow scenario everything i highly recommend mystery lovers anyone wants watch good movie']


In [29]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[  78  690  171 1231  127    3   14 2739 2637  171    1  470  278  739
  1663  152  392   33    8    3    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]]


In [30]:
result = loaded_model.predict(tokenize_words)
print(result)

[[0.99531007]]


In [34]:
if result >=0.5 :
    print('positive')
else:
    print('negative')

positive
